## PHASE 2 PROJECT.
### GROUP 2.
#### Members





#### Business Problem.
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.


From a pool of Objectives offer three solid recommendations to the firm.

### Objectives.
1. Understand runtime trends considering ratings. ( GODFREY)
2. Potential movie directors that have the highest audience ratings. (JOY)
3. Studios with the highest returns. (JOY)
4. Production budget that yields the most profit return and Evaluate the Impact of Budget on Success. (Godfrey)
5. Compare between the worldwide and domestic patterns which one has more profitable market. (Emmanuel)
6. Understand Genre Performance in terms of ratings (Faith)
7. Identify Profitable Films. (Faith)
8. Find Seasonal or Time-Based Trends (Emmanuel)


In [ ]:
### OBJECTIVE 1. 